In [26]:
import requests
from bs4 import BeautifulSoup as bs
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

## archives.gov electoral vote scraper

In [ ]:
def alfa(x): return ''.join([c for c in x if c.isalpha() or c==' '])

In [ ]:
elec_vote_map = dict()

for y in range(1976, 2021, 4):
    uri = f'https://www.archives.gov/electoral-college/{y}'
    r = requests.get(uri)
    states = bs(r.text, 'html.parser').findAll('tbody')[1].findAll('tr')
    for i in range(2,53):
        cell_list = states[i].findAll('td')

        s = alfa(cell_list[0].text).upper()
        ev = int(cell_list[1].text)
        elec_vote_map[(y,s)] = ev

pickle.dump(elec_vote_map, open('datasets/elec_vote_map.pkl', 'wb'))

In [27]:
# solo sulla percentuale, non sulla assegnazione
df = pd.read_csv('datasets/fair-fitted-values.csv')
rms = np.sqrt(mean_squared_error(df.y_votes_percent, df.y_hat_vote))

In [ ]:
df = pd.read_csv('datasets/fair-fitted-values.csv')
df = df[['year', 'state', 'y_votes_percent', 'y_hat_vote', ]]
for i in df.index:
    q = df.at[i, 'y_votes_percent']
    p = df.at[i, 'y_hat_vote']
    y = df.at[i, 'year']
    s = df.at[i, 'state']

    df.at[i, 'y_votes_percent'] = elec_vote_map[(y,s)] if q > 50 else 0
    df.at[i, 'y_hat_vote'] = elec_vote_map[(y,s)] if p > 50 else 0


In [ ]:
# DEM EL VOTE
df.groupby('year')['y_votes_percent', 'y_hat_vote',].sum()